In [135]:
import folium
import zipfile
import io

from fastkml import kml

import pandas as pd
from io import StringIO

import csv
from tqdm import tqdm

In [2]:
# read KML content 
kmz = zipfile.ZipFile("qfaults.kmz", 'r')  # extract zip file first, then read kmz file inside the extracted folder
kml_content = kmz.open('doc.kml', 'r').read()  # kml content
k = kml.KML()
k.from_string(kml_content)

In [65]:
docs = list(k.features())
folders=[]
for d in docs:
    folders.extend(list(d.features()))
records = []
for f in folders:
    records.extend(list(f.features()))
geoms = [element.geometry for element in records]

In [78]:
# print(dir(folders[0]))
print(folders[0].name)
print(folders[0].id)

Historical (150 years)
FeatureLayer0


In [20]:
names = [element.name for element in records]
print(len(set(names)))
print(len(name))

2107
112527


In [130]:
def get_header(record):
    tables = pd.read_html(StringIO(str(record.description)))
    data_table = list(tables[1][0])
    return data_table

def read_record(record):
    tables = pd.read_html(StringIO(str(record.description)))
    data_table = list(tables[1][1])
    return data_table

def get_all_data(record):
    data = []
    data.append(record.name)
    data.append(record.id)
    data.extend(read_record(record))
    data.append(record.geometry.wkt)
    return data

def get_all_headers(record):
    data = []
    data.append("Name")
    data.append("ID")
    data.extend(get_header(record))
    data.append("Geom")
    return data

print(get_all_headers(records[0]))
print(get_all_data(records[0]))
# headers = [len(get_header(element)) for element in records]

['Name', 'ID', 'Fault Name', 'Section Name', 'Fault ID', 'Section ID', 'Location', 'Linetype', 'Age', 'Dip Direction', 'Slip Rate(mm/year)', 'Slip Sense', 'Mapped Scale', 'Fault Class', 'Mapping Certainty', 'Average Strike', 'Total Fault Length (km)', 'Cooperator', 'Associated Earthquake', 'Date of Last Review', 'Fault url', 'symbology', 'Citation ID', 'Fault ID Number', 'Geom']
['San Andreas fault zone', 'ID_00000', 'San Andreas fault zone', 'Shelter Cove Section', '1', 'a', 'California', 'Inferred', 'historic', 'Vertical', 'Greater than 5.0 mm/yr', 'Right lateral', 'unspecified', 'A', 'Good', 'N12°W', '1082', 'California Geological Survey', 'San Francisco earthquake', '12/10/2002', 'https://earthquake.usgs.gov/cfusion/qfault/show_report_AB_archive.cfm?fault_id=1&section_id=a', 'historic Inferred', '1a', '1', 'MULTILINESTRING((-124.090694000324 40.11756400000441 0.0, -124.0905649995543 40.11569200027037 0.0, -124.0904559997744 40.11512000009119 0.0, -124.0901240004117 40.1146650003225

In [131]:
geoms = [element.geometry for element in records]  # extract geometry
# dir(geoms[0])

In [ ]:
filename = 'Qfaults.csv'
with open(filename, 'w', newline="") as file:
    csvwriter = csv.writer(file)
    header = get_all_headers(records[0])
    csvwriter.writerow(header)
    for record in tqdm (records, desc="Writing File..."):
        data = get_all_data(record)
        csvwriter.writerow(data)
    # csvwriter.writerows(data) # 5. write the rest of the data

Writing File...:  99%|███████████████████████████████████████████████████████▌| 111742/112527 [05:00<00:02, 368.05it/s]